In [1]:
import pandas

df = pandas.read_csv("voted_df.csv")

In [2]:
df

,videoID,startTime,endTime,votes
0,fBxtS9BpVWs,41.00000,53.000000,116
1,fBxtS9BpVWs,714.00000,763.000000,64
2,uqKGREZs6-w,475.52167,532.208740,302
3,1rb3bMvDdX4,588.94100,631.897770,158
4,1rb3bMvDdX4,0.00000,6.861597,348
...,...,...,...,...
122708,idX-x5W5O30,35.17800,121.273000,4
122709,6gi6SZyMGiA,208.10900,266.010000,3
122710,bFNJVaO9E-o,66.15000,80.957000,7
122711,bFNJVaO9E-o,697.50000,732.581000,5


In [17]:
from collections import defaultdict
vid_to_sponsors = defaultdict(list)

for row in df.itertuples():
    vid_to_sponsors[row.videoID].append((row.startTime, row.endTime))

vid_to_sponsors.default_factory = None
len(vid_to_sponsors)

85349

In [3]:
import json

def get_transcript(vid):
    return json.load(open(f"transcripts_merged/{vid}.json"))

In [32]:
import bisect

def process_vid(vid: str):
    transcript = get_transcript(vid)
    segments = vid_to_sponsors[vid]
    for start, end in segments:
        start *= 1000
        end *= 1000

        idx_l = bisect.bisect(transcript, start, key=lambda x: x["s"])
        idx_r = bisect.bisect(transcript, end, key=lambda x: x["e"])

        for i in range(idx_l, idx_r):
            transcript[i]["m"] = True

    tokens = []
    token_class = []
    for item in transcript:
        tokens.append(item["w"].strip())
        if "m" in item:
            token_class.append("Y")
        else:
            token_class.append("N")
    
    return tokens, token_class

In [39]:
import glob
from tqdm import tqdm
from os import path

final_data = []
skipped = 0
for filepath in tqdm(glob.iglob('transcripts_merged/*.json'), total=len(list(glob.iglob('transcripts_merged/*.json')))):
    vid = path.basename(filepath).split(".")[0]
    try:
        tokens, tags = process_vid(vid)
    except TypeError:
        skipped += 1
    except Exception as e:
        skipped += 1
    final_data.append({"tokens": tokens, "tags": tags})

print(f"Skipped {skipped} videos")

100%|██████████| 31608/31608 [01:14<00:00, 425.68it/s]

Skipped 201 videos


In [40]:
import json

with open("data.jsonl", "w") as outfile:
    for entry in final_data:
        json.dump(entry, outfile)
        outfile.write('\n')